In [ ]:


def lineNotifyMessage(token, msg):
    import requests
    headers = {
       "Authorization": "Bearer " + token,
       "Content-Type" : "application/x-www-form-urlencoded"
    }
    payload = {'message': msg}
    r = requests.post("https://notify-api.line.me/api/notify", headers = headers, params = payload)
    return r.status_code

def main(分點,date = date,stock = stock):
    params = (date,stock)
    conn = pymssql.connect(host = 'localhost')
    cursor = conn.cursor()
    querystr = 'SELECT  [scode],[sname],成交量,[成交金額(千)],收盤價 FROM [MyDB1].[dbo].[CMoney_日收盤表排行] where txn=%s and scode = %s'
    cursor.execute(querystr,params=params)
    data = []
    for row in cursor:
        temp = []
        for i in row:
            temp.append(i)
           # print(temp)
        data.append(temp)
    data = pd.DataFrame(data,columns=['公司代號','公司名稱','成交量','成交金額(千)','收盤價'])
    params = [date,stock,分點]
    conn = pymssql.connect(host = 'localhost')
    cursor = conn.cursor()
    select = 'SELECT  txn,[scode],[sname],[bcode],[bname],sum([買張]-[賣張])as [買超(張)]'
    From = 'FROM [MyDB1].[dbo].[CMoney_個股券商分點進出明細]'
    Where = ' where txn = %s and scode = %s and bname in %s'
    groupby = 'group by sname, scode, bcode, bname,txn'
    querystr = select+From+Where+groupby
    cursor.execute(querystr,params=(date,stock,分點))
    data1 = []
    for row in cursor:
        temp = []
        for i in row:
            temp.append(i)
        data1.append(temp)
    data1 = pd.DataFrame(data1,columns = ['date','公司代號','公司名稱','分點代號','分點名稱','買超'])
    data1['成交量'] = data['成交量'].values[0]
    if data1.empty :
        pass
    elif data1['買超'].cumsum().iloc[-1]> data1['成交量'].values[0]* 0.2 :
        token = 'iNHGpwf7KAau4gPA2nIFuBG00B5Z9mqw6TDPs0ulrxM'
        message = f'''
    ⏰日期:{date}
    🆔公司代號: {stock}
    📦公司名稱: {data1['公司名稱'].values[0]}
    💹地緣買超占比: {round(data1['買超'].cumsum().iloc[-1] / data1['成交量'].values[0] * 100, 2)}%
    📊總成交量: {data1['成交量'].values[0]}
    💰今日收盤價: {data['收盤價'].values[0]}
    '''
    lineNotifyMessage(token,message)